# This notebook shows how to manage the events table with Spark

We first need to the kudu jars to the spark session

In [1]:
%%configure -f
{ "conf": {"spark.jars.packages": "org.apache.kudu:kudu-spark2_2.11:1.4.0" }}

In [2]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
34,application_1501855380192_0154,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4ebdbcaf

In [3]:
import org.apache.kudu.spark.kudu._
import org.apache.spark.sql.functions._

//we need to define a udf function for converting two strings into a map
val asMap = udf((keys:String, values: String) => keys.split("#").zip(values.split("#")).toMap)

//then we define a first dataframe df1 associated to the kudu table
val df1 = spark.sqlContext.read.options(Map("kudu.master" -> "master:7051","kudu.table" -> "Events")).kudu

//the next dataframe contains an additional column of type Map containing a map for the two arrays: keys and values
val df2 = df1.withColumn("tags", asMap($"attributesKeys",$"attributesValues"))

df2: org.apache.spark.sql.DataFrame = [id: string, ts: bigint ... 12 more fields]

In [4]:
//let's show a couple of rows of df2
df2.select("id", "ts", "tags").take(10).toList

res9: List[org.apache.spark.sql.Row] = List()

In [5]:
df2.count()

res10: Long = 0

In [6]:
%%cleanup -f